"""
Created on: Wed Jan 15 15:57:30 2020
Group 7
@author: C.T.D.
"""

In [83]:

import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
import numpy as np
import pandas as pd
import math
import operator
import os
from PIL import Image
from tqdm import tqdm_notebook as tqdm
import random
import pickle

import shutil
from shutil import copyfile

# keras : librairie de deep learning
import keras
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, InputLayer, ReLU, AveragePooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

# import for transfer learning
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

In [84]:
# Reading of csv file SEATGURU annontate
csv_file = pd.read_csv('../g7_SEATGURU_annotate.csv', sep=';')
Airbus_file = csv_file.loc[csv_file['aircraft_manufacturer']  == 'Airbus',:]
Airbus_file = Airbus_file.loc[Airbus_file['view'] == 'Int']
Boeing_file = csv_file.loc[csv_file['aircraft_manufacturer']  == 'Boeing',:]
Boeing_file = Boeing_file.loc[Boeing_file['view'] == 'Int']
# List name pictures
list_img_Airbus = list(Airbus_file['name'])

A320_file = Airbus_file.loc[Airbus_file['aircraft_type'] == 'A320']
list_img_A320 = list(A320_file['name'])
A321_file = Airbus_file.loc[Airbus_file['aircraft_type'] == 'A321']
list_img_A321 = list(A321_file['name'])
A330_file = Airbus_file.loc[Airbus_file['aircraft_type'] == 'A330']
list_img_A330 = list(A330_file['name'])
A350_file = Airbus_file.loc[Airbus_file['aircraft_type'] == 'A350']
list_img_A350 = list(A350_file['name'])
A380_file = Airbus_file.loc[Airbus_file['aircraft_type'] == 'A380']
list_img_A380 = list(A380_file['name'])

list_img_Boeing = list(Boeing_file['name'])

B737_file = Boeing_file.loc[Boeing_file['aircraft_type'] == '737']
list_img_B737 = list(B737_file['name'])
B767_file = Boeing_file.loc[Boeing_file['aircraft_type'] == '767']
list_img_B767 = list(B767_file['name'])
B777_file = Boeing_file.loc[Boeing_file['aircraft_type'] == '777']
list_img_B777 = list(B777_file['name'])
B787_file = Boeing_file.loc[Boeing_file['aircraft_type'] == '787']
list_img_B787 = list(B787_file['name'])



In [85]:
path_seatguru = './../All Data/ANALYSE IMAGE/IMG SEATGURU/'
path_hackathon = './../Inputs Hackathon/'
new_path_train = './../Split_data/Hack_Seatguru/Int/data_train'
new_path_val = './../Split_data/Hack_Seatguru/Int/data_val'
new_paths = [new_path_train, new_path_val]

In [86]:
# function to split data in train and test samples for Seatguru pictures
def split_train_test_Seatguru_cat(new_paths, path, 
                                  airbus_planes = [list_img_A320, list_img_A321, list_img_A330,list_img_A350, list_img_A380], 
                                  boeing_planes = [list_img_B737,list_img_B767,list_img_B777,list_img_B787],
                                  split_limit = .7, s = 8):
  
    #Boeing

    for plane in boeing_planes:
        if plane == list_img_B737:
            classe = '737'
        if plane == list_img_B767:
            classe = '767'
        if plane == list_img_B777:
            classe = '777'
        if plane == list_img_B787:
            classe = '787'
        
        shutil.rmtree(new_paths[0] + '/' + classe, ignore_errors = True)
        os.makedirs(new_paths[0] + '/' + classe, exist_ok = True)
        shutil.rmtree(new_paths[1] + '/' + classe, ignore_errors = True)
        os.makedirs(new_paths[1]+ '/' + classe, exist_ok = True)
        
        random.seed(a = s)
        random.shuffle(plane)

        for pict in plane[:int(split_limit*len(plane))]:
            copyfile(path + '/' + pict, new_paths[0] + '/' + classe + '/' +  pict)
        for pict in plane[int(split_limit*len(plane)):]:
            copyfile(path + '/' + pict, new_paths[1] + '/' + classe + '/' +  pict)
            
    #Airbus
        
    for plane in airbus_planes:

        if plane == list_img_A320:
            classe = 'A320'
        if plane == list_img_A321:
            classe = 'A321'
        if plane == list_img_A330:
            classe = 'A330'
        if plane == list_img_A350:
            classe = 'A350'
        if plane == list_img_A380:
            classe = 'A380'
        
        shutil.rmtree(new_paths[0] + '/' + classe, ignore_errors = True)
        os.makedirs(new_paths[0] + '/' + classe, exist_ok = True)
        shutil.rmtree(new_paths[1] + '/' + classe, ignore_errors = True)
        os.makedirs(new_paths[1]+ '/' + classe, exist_ok = True)
        
        random.seed(a = s)
        random.shuffle(plane)

        for pict in plane[:int(split_limit*len(plane))]:
            copyfile(path + '/' + pict, new_paths[0] + '/' + classe + '/' +  pict)
        for pict in plane[int(split_limit*len(plane)):]:
            copyfile(path + '/' + pict, new_paths[1] + '/' + classe + '/' +  pict)

In [87]:

# function to split data in train and test samples for Hackathon pictures
def  split_train_test_Hack_cat (new_paths:list, path:str, airbus_planes:list, split_limit:float=.7):
    
    #Airbus
    
    for plane in airbus_planes:
        
        os.makedirs(new_paths[0] + '/' + plane, exist_ok = True)
        os.makedirs(new_paths[1] + '/' + plane, exist_ok = True)
        
        for pict in os.listdir(path + plane):
            rand = np.random.random()
            if rand <= split_limit:
                copyfile(path + plane + '/' + pict, new_paths[0] + '/' + plane + '/' + pict)
            else:
                copyfile(path + plane + '/' + pict, new_paths[1] + '/' + plane + '/' + pict)
                

In [88]:
# Creation of train and test samples for Seatguru pictures
split_train_test_Seatguru_cat(new_paths, path_seatguru)

In [89]:
# Creation of train and test samples for Hackathon pictures
split_train_test_Hack_cat(new_paths, path_hackathon, airbus_planes = ['A320', 'A330', 'A350', 'A380'])

In [90]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

In [91]:
train_generator = train_datagen.flow_from_directory(new_path_train,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 1766 images belonging to 9 classes.


In [92]:
train_generator.class_indices

{'737': 0,
 '767': 1,
 '777': 2,
 '787': 3,
 'A320': 4,
 'A321': 5,
 'A330': 6,
 'A350': 7,
 'A380': 8}

In [93]:
test_generator = train_datagen.flow_from_directory(new_path_val,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 747 images belonging to 9 classes.


In [ ]:
#Deep Learning

In [98]:
def create_model(input_shape = (224, 224, 3), nb_couches = 4, kernel = (1,1), pool = (2, 2), nb_classes = 9):
    model = Sequential()
    
    model.add(InputLayer(input_shape=input_shape))
    
    for i in range(1,5):
        model.add(Conv2D(3**i, kernel_size=kernel))
        model.add(keras.layers.BatchNormalization())
        model.add(ReLU())

        model.add(Conv2D(3**i, kernel_size=kernel))
        model.add(keras.layers.BatchNormalization())
        model.add(ReLU())
        
        model.add(Conv2D(3**i, kernel_size=kernel))
        model.add(keras.layers.BatchNormalization())
        model.add(ReLU())

        model.add(Conv2D(3**i, kernel_size=kernel))
        model.add(keras.layers.BatchNormalization())
        model.add(ReLU())
        
        model.add(Conv2D(3**i, kernel_size=kernel))
        model.add(keras.layers.BatchNormalization())
        model.add(ReLU())

        model.add(MaxPooling2D(pool_size=pool))
    
        
    model.add(Conv2D(2, kernel_size=kernel))
    model.add(keras.layers.BatchNormalization())
    model.add(ReLU())        
    
    model.add(Conv2D(2, kernel_size=kernel))
    model.add(keras.layers.BatchNormalization())
    model.add(ReLU())        
    
    model.add(Conv2D(2, kernel_size=kernel))
    model.add(keras.layers.BatchNormalization())
    model.add(ReLU())        
    
    model.add(Conv2D(2, kernel_size=kernel))
    model.add(keras.layers.BatchNormalization())
    model.add(ReLU()) 
    
    model.add(Conv2D(2, kernel_size=kernel))
    model.add(keras.layers.BatchNormalization())
    model.add(ReLU()) 
    
    model.add(MaxPooling2D(pool_size=pool))    
        
    for i in range(4,2,-1):
        model.add(Conv2D(3**i, kernel_size=kernel))
        model.add(keras.layers.BatchNormalization())
        model.add(ReLU())

        model.add(Conv2D(3**i, kernel_size=kernel))
        model.add(keras.layers.BatchNormalization())
        model.add(ReLU())
        
        model.add(Conv2D(3**i, kernel_size=kernel))
        model.add(keras.layers.BatchNormalization())
        model.add(ReLU())

        model.add(Conv2D(3**i, kernel_size=kernel))
        model.add(keras.layers.BatchNormalization())
        model.add(ReLU())
               
        model.add(Conv2D(3**i, kernel_size=kernel))
        model.add(keras.layers.BatchNormalization())
        model.add(ReLU())

        model.add(MaxPooling2D(pool_size=pool))

    model.add(Flatten())

    model.add(Dense(512))
    model.add(keras.layers.BatchNormalization())
    model.add(ReLU())
    
    model.add(Dense(1024))
    model.add(keras.layers.BatchNormalization())
    model.add(ReLU())
    
    model.add(Dense(nb_classes, activation='softmax'))

    model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')

    return model

In [99]:
model = create_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_36 (Conv2D)           (None, 224, 224, 3)       12        
_________________________________________________________________
batch_normalization_38 (Batc (None, 224, 224, 3)       12        
_________________________________________________________________
re_lu_38 (ReLU)              (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 224, 224, 3)       12        
_________________________________________________________________
batch_normalization_39 (Batc (None, 224, 224, 3)       12        
_________________________________________________________________
re_lu_39 (ReLU)              (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 224, 224, 3)      

conv2d_55 (Conv2D)           (None, 28, 28, 81)        6642      
_________________________________________________________________
batch_normalization_57 (Batc (None, 28, 28, 81)        324       
_________________________________________________________________
re_lu_57 (ReLU)              (None, 28, 28, 81)        0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 14, 14, 81)        0         
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 14, 14, 2)         164       
_________________________________________________________________
batch_normalization_58 (Batc (None, 14, 14, 2)         8         
_________________________________________________________________
re_lu_58 (ReLU)              (None, 14, 14, 2)         0         
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 14, 14, 2)         6         
__________

In [100]:
#Sans reduction d'apprentissage si necessaire

In [101]:
step_size_train=train_generator.n//train_generator.batch_size

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                              patience=1, 
                              verbose=1, 
                              factor=0.5, 
                              min_lr=0.00001)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=5,
                   validation_data = test_generator)

Epoch 1/5
55/55 [==============================] - ETA: 20:44 - loss: 0.4303 - accuracy: 0.878 - ETA: 12:00 - loss: 0.4033 - accuracy: 0.880 - ETA: 9:00 - loss: 0.4433 - accuracy: 0.875 - ETA: 7:24 - loss: 0.4497 - accuracy: 0.87 - ETA: 6:26 - loss: 0.4435 - accuracy: 0.87 - ETA: 5:43 - loss: 0.4474 - accuracy: 0.87 - ETA: 5:12 - loss: 0.4437 - accuracy: 0.86 - ETA: 4:46 - loss: 0.4390 - accuracy: 0.86 - ETA: 4:26 - loss: 0.4329 - accuracy: 0.87 - ETA: 4:09 - loss: 0.4296 - accuracy: 0.87 - ETA: 3:54 - loss: 0.4257 - accuracy: 0.87 - ETA: 3:41 - loss: 0.4254 - accuracy: 0.87 - ETA: 3:30 - loss: 0.4234 - accuracy: 0.87 - ETA: 3:20 - loss: 0.4241 - accuracy: 0.87 - ETA: 3:11 - loss: 0.4232 - accuracy: 0.87 - ETA: 3:03 - loss: 0.4212 - accuracy: 0.87 - ETA: 2:56 - loss: 0.4196 - accuracy: 0.87 - ETA: 2:49 - loss: 0.4190 - accuracy: 0.87 - ETA: 2:42 - loss: 0.4194 - accuracy: 0.87 - ETA: 2:36 - loss: 0.4175 - accuracy: 0.87 - ETA: 2:30 - loss: 0.4163 - accuracy: 0.87 - ETA: 2:24 - loss: 0.

55/55 [==============================] - ETA: 2:53 - loss: 0.3647 - accuracy: 0.88 - ETA: 2:55 - loss: 0.3585 - accuracy: 0.88 - ETA: 3:00 - loss: 0.3591 - accuracy: 0.88 - ETA: 3:03 - loss: 0.3521 - accuracy: 0.88 - ETA: 3:04 - loss: 0.3509 - accuracy: 0.88 - ETA: 3:01 - loss: 0.3446 - accuracy: 0.88 - ETA: 2:59 - loss: 0.3458 - accuracy: 0.88 - ETA: 2:58 - loss: 0.3456 - accuracy: 0.88 - ETA: 2:56 - loss: 0.3460 - accuracy: 0.88 - ETA: 2:52 - loss: 0.3449 - accuracy: 0.88 - ETA: 2:49 - loss: 0.3472 - accuracy: 0.88 - ETA: 2:45 - loss: 0.3510 - accuracy: 0.88 - ETA: 2:40 - loss: 0.3506 - accuracy: 0.88 - ETA: 2:35 - loss: 0.3511 - accuracy: 0.88 - ETA: 2:32 - loss: 0.3494 - accuracy: 0.88 - ETA: 2:27 - loss: 0.3486 - accuracy: 0.88 - ETA: 2:23 - loss: 0.3486 - accuracy: 0.88 - ETA: 2:20 - loss: 0.3500 - accuracy: 0.88 - ETA: 2:16 - loss: 0.3512 - accuracy: 0.88 - ETA: 2:11 - loss: 0.3517 - accuracy: 0.88 - ETA: 2:07 - loss: 0.3511 - accuracy: 0.88 - ETA: 2:03 - loss: 0.3505 - accuracy